# Phase 10: Bear Market Profit Backtest

Test the selective shorting strategy for bear market profits:
- Bear mode triggers on high ATR >5% + RSI overbought >70
- Deploys Kraken 3-5x shorts on XRP, Bitrue 3x short ETFs (BTC3S) on BTC
- Short decay penalty after 14 days forces timely exits
- Goal: Flip Phase 9's -9.2% to flat/positive via downside capture

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_fetcher import DataFetcher
from portfolio import Portfolio
from orchestrator import RLOrchestrator
from risk_manager import RiskManager
from strategies.dip_buy_lstm import generate_dip_signals, detect_dip_signal

print("Phase 10: Bear Market Profit Backtest")
print("="*50)

## 1. Fetch Historical Data (Nov-Dec Period)

In [ ]:
fetcher = DataFetcher()

# Fetch extended data to capture bear periods (Nov-Dec)
data = {}
for sym in ['XRP/USDT', 'BTC/USDT']:
    print(f"Fetching {sym}...")
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 2000)
    if not df.empty:
        data[sym] = df
        print(f"  {len(df)} candles from {df.index[0]} to {df.index[-1]}")

# Show price statistics
if 'XRP/USDT' in data:
    xrp = data['XRP/USDT']
    print(f"\nXRP range: ${xrp['close'].min():.4f} - ${xrp['close'].max():.4f}")
    print(f"XRP current: ${xrp['close'].iloc[-1]:.4f}")
    
if 'BTC/USDT' in data:
    btc = data['BTC/USDT']
    print(f"\nBTC range: ${btc['close'].min():.2f} - ${btc['close'].max():.2f}")
    print(f"BTC current: ${btc['close'].iloc[-1]:.2f}")

## 2. Identify Bear Market Opportunities (Shorting Signals)

In [ ]:
risk = RiskManager(max_drawdown=0.20, max_leverage=10.0)

def calculate_rsi(close, period=14):
    """Calculate RSI."""
    if len(close) < period + 1:
        return 50.0
    deltas = np.diff(close[-period-1:])
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    avg_gain = np.mean(gains) if len(gains) > 0 else 0
    avg_loss = np.mean(losses) if len(losses) > 0 else 0.001
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Find bear/short opportunities in XRP
if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    bear_signals = []
    for i in range(50, len(df) - 24):  # Leave room for outcome check
        close = df['close'].iloc[:i+1].values
        volume = df['volume'].iloc[:i+1].values
        high = df['high'].iloc[:i+1].values
        low = df['low'].iloc[:i+1].values
        
        # Calculate indicators
        rsi = calculate_rsi(close)
        atr_pct = risk.calculate_atr_pct(high, low, close)
        
        # Check for bear setup: RSI overbought + high ATR
        is_bear_setup = (
            rsi > 70 and  # Overbought
            atr_pct > 0.05  # High volatility (bear regime)
        )
        
        if is_bear_setup:
            # Check if drop happened (price down 5%+ in next 24h)
            future_prices = df['close'].iloc[i:i+24].values
            current_price = close[-1]
            min_future = np.min(future_prices)
            drop_pct = (current_price - min_future) / current_price * 100
            
            bear_signals.append({
                'timestamp': df.index[i],
                'price': current_price,
                'rsi': rsi,
                'atr_pct': atr_pct * 100,
                'drop_pct': drop_pct,
                'success': drop_pct > 5  # 5% drop = successful short
            })
    
    bear_df = pd.DataFrame(bear_signals)
    print(f"\nFound {len(bear_signals)} bear/short setups")
    
    if len(bear_df) > 0:
        success_rate = bear_df['success'].mean() * 100
        avg_drop = bear_df['drop_pct'].mean()
        print(f"Success rate (>5% drop): {success_rate:.1f}%")
        print(f"Average drop: {avg_drop:.2f}%")
        
        print(f"\nHigh RSI bear signals (>75):")
        high_rsi = bear_df[bear_df['rsi'] > 75]
        if len(high_rsi) > 0:
            print(high_rsi.to_string())
            high_rsi_success = high_rsi['success'].mean() * 100
            print(f"\nHigh-RSI success rate: {high_rsi_success:.1f}%")

## 3. Bear vs Greed Regime Analysis

In [ ]:
if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    # Calculate rolling indicators
    indicators = []
    for i in range(50, len(df)):
        close = df['close'].iloc[:i+1].values
        high = df['high'].iloc[:i+1].values
        low = df['low'].iloc[:i+1].values
        
        rsi = calculate_rsi(close)
        atr_pct = risk.calculate_atr_pct(high, low, close)
        
        # Determine regime (Phase 10: added bear)
        if atr_pct > 0.05 and rsi > 70:
            regime = 'bear'
        elif atr_pct < 0.02 and rsi < 30:
            regime = 'greed'
        elif atr_pct > 0.05:
            regime = 'fear'
        else:
            regime = 'neutral'
        
        indicators.append({
            'timestamp': df.index[i],
            'price': close[-1],
            'rsi': rsi,
            'atr_pct': atr_pct * 100,
            'regime': regime
        })
    
    ind_df = pd.DataFrame(indicators)
    ind_df.set_index('timestamp', inplace=True)
    
    # Plot
    fig, axes = plt.subplots(4, 1, figsize=(14, 12))
    
    # Price
    axes[0].plot(ind_df.index, ind_df['price'], label='XRP Price')
    axes[0].set_title('XRP/USDT Price')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    
    # RSI with overbought zone highlighted
    axes[1].plot(ind_df.index, ind_df['rsi'], label='RSI', color='purple')
    axes[1].axhline(y=30, color='green', linestyle='--', label='Oversold (Long)')
    axes[1].axhline(y=70, color='red', linestyle='--', label='Overbought (Short)')
    axes[1].fill_between(ind_df.index, 0, 30, alpha=0.2, color='green')
    axes[1].fill_between(ind_df.index, 70, 100, alpha=0.2, color='red')
    axes[1].set_title('RSI Indicator (Phase 10: >70 = Short Signal)')
    axes[1].set_ylabel('RSI')
    axes[1].legend()
    
    # ATR%
    axes[2].plot(ind_df.index, ind_df['atr_pct'], label='ATR %', color='orange')
    axes[2].axhline(y=2, color='green', linestyle='--', label='Greed Threshold')
    axes[2].axhline(y=5, color='red', linestyle='--', label='Bear Threshold')
    axes[2].fill_between(ind_df.index, 0, 2, alpha=0.2, color='green')
    axes[2].fill_between(ind_df.index, 5, ind_df['atr_pct'].max(), alpha=0.2, color='red')
    axes[2].set_title('Volatility (ATR %) - Phase 10: >5% = Bear Mode')
    axes[2].set_ylabel('ATR %')
    axes[2].legend()
    
    # Regime (Phase 10: added bear)
    regime_map = {'greed': 1, 'neutral': 0, 'fear': -0.5, 'bear': -1}
    regime_vals = ind_df['regime'].map(regime_map)
    colors = ['green' if r == 'greed' else 'darkred' if r == 'bear' else 'red' if r == 'fear' else 'gray' for r in ind_df['regime']]
    axes[3].bar(ind_df.index, regime_vals, color=colors, alpha=0.6, width=0.02)
    axes[3].set_title('Market Regime (1=Greed/Long, -1=Bear/Short, -0.5=Fear/Defensive)')
    axes[3].set_ylabel('Regime')
    axes[3].set_ylim(-1.5, 1.5)
    
    plt.tight_layout()
    plt.show()
    
    # Regime stats
    print(f"\nRegime Distribution:")
    print(ind_df['regime'].value_counts())
    print(f"\n% Time in Greed (Long): {(ind_df['regime'] == 'greed').mean() * 100:.1f}%")
    print(f"% Time in Bear (Short): {(ind_df['regime'] == 'bear').mean() * 100:.1f}%")
    print(f"% Time in Fear (Park):  {(ind_df['regime'] == 'fear').mean() * 100:.1f}%")

## 4. Phase 10 Backtest - Bear Profit Strategy

In [ ]:
# Initialize portfolio and orchestrator
starting = {'USDT': 1000.0, 'XRP': 500.0, 'BTC': 0.0}
portfolio = Portfolio(starting.copy())

try:
    orchestrator = RLOrchestrator(portfolio, data)
    rl_enabled = orchestrator.enabled
    print(f"\nRL Orchestrator: {'Enabled' if rl_enabled else 'Disabled'}")
    if rl_enabled:
        print(f"  Targets: {orchestrator.get_target_allocation()}")
        print(f"  Bear confidence threshold: {orchestrator.BEAR_CONFIDENCE_THRESHOLD}")
        print(f"  Bear vol threshold: {orchestrator.BEAR_VOL_THRESHOLD * 100}%")
        print(f"  RSI overbought: {orchestrator.RSI_OVERBOUGHT}")
except Exception as e:
    print(f"Could not initialize orchestrator: {e}")
    rl_enabled = False

In [ ]:
if rl_enabled and 'XRP/USDT' in data:
    df = data['XRP/USDT']
    btc_df = data.get('BTC/USDT')
    
    # Backtest loop
    results = []
    actions_log = []
    short_log = []  # Phase 10: Track shorts
    
    step_size = 4  # Every 4 hours
    
    for i in range(100, len(df), step_size):
        timestamp = df.index[i]
        xrp_price = df['close'].iloc[i]
        btc_price = btc_df['close'].iloc[i] if btc_df is not None and i < len(btc_df) else 90000.0
        
        prices = {'XRP': xrp_price, 'BTC': btc_price, 'USDT': 1.0}
        
        # Update env step
        if orchestrator.env is not None:
            orchestrator.env.current_step = min(i, orchestrator.env.max_steps - 1)
        
        # Get RL decision
        result = orchestrator.decide_and_execute(prices)
        orchestrator.check_and_manage_positions(prices)
        orchestrator.update_env_step()
        
        # Log
        total_value = portfolio.get_total_usd(prices)
        results.append({
            'timestamp': timestamp,
            'xrp_price': xrp_price,
            'btc_price': btc_price,
            'portfolio_value': total_value,
            'usdt': portfolio.balances.get('USDT', 0),
            'xrp': portfolio.balances.get('XRP', 0),
            'btc': portfolio.balances.get('BTC', 0),
            'volatility': result.get('volatility', 0),
            'mode': result.get('mode', 'unknown'),
            'market_state': result.get('market_state', 'unknown'),
            'rsi_xrp': result.get('rsi', {}).get('XRP', 50),
            'rsi_btc': result.get('rsi', {}).get('BTC', 50)
        })
        
        if result.get('executed'):
            action_entry = {
                'timestamp': timestamp,
                'action': f"{result['asset']} {result['action_type']}",
                'leverage': result.get('leverage', 0),
                'mode': result.get('mode', 'unknown'),
                'trigger': result.get('offensive_trigger', result.get('short', 'standard')),
                'price': xrp_price,
                'confidence': result.get('confidence', 0)
            }
            actions_log.append(action_entry)
            
            # Phase 10: Log shorts specifically
            if result.get('short'):
                short_log.append({
                    'timestamp': timestamp,
                    'asset': result['short'],
                    'entry_price': result.get('collateral', 0),
                    'leverage': result.get('leverage', 0),
                    'collateral': result.get('collateral', 0)
                })
    
    results_df = pd.DataFrame(results)
    results_df.set_index('timestamp', inplace=True)
    
    print(f"\nBacktest complete: {len(results)} data points")
    print(f"Actions executed: {len(actions_log)}")
    print(f"Short positions opened: {len(short_log)}")

In [ ]:
if 'results_df' in dir() and len(results_df) > 0:
    # Plot results
    fig, axes = plt.subplots(5, 1, figsize=(14, 16))
    
    # Portfolio value
    axes[0].plot(results_df.index, results_df['portfolio_value'], label='Portfolio Value', linewidth=2)
    axes[0].set_title('Phase 10: Portfolio Value - Bear Profit Strategy')
    axes[0].set_ylabel('Value ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Holdings
    axes[1].plot(results_df.index, results_df['usdt'], label='USDT', color='green')
    axes[1].plot(results_df.index, results_df['xrp'] * results_df['xrp_price'], label='XRP (USD)', color='blue')
    axes[1].plot(results_df.index, results_df['btc'] * results_df['btc_price'], label='BTC (USD)', color='orange')
    axes[1].set_title('Holdings (USD Value)')
    axes[1].set_ylabel('Value ($)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Trading mode (Phase 10: includes bear)
    mode_colors = results_df['mode'].map({'offensive': 'green', 'defensive': 'orange', 'bear': 'red', 'unknown': 'gray'})
    mode_vals = results_df['mode'].map({'offensive': 1, 'defensive': 0, 'bear': -1, 'unknown': 0})
    axes[2].bar(results_df.index, mode_vals, color=mode_colors.values, alpha=0.6, width=0.02)
    axes[2].set_title('Trading Mode (1=Offensive/Long, 0=Defensive, -1=Bear/Short)')
    axes[2].set_ylabel('Mode')
    axes[2].set_ylim(-1.5, 1.5)
    
    # RSI with short zone
    axes[3].plot(results_df.index, results_df['rsi_xrp'], label='XRP RSI', color='blue')
    axes[3].plot(results_df.index, results_df['rsi_btc'], label='BTC RSI', color='orange')
    axes[3].axhline(y=30, color='green', linestyle='--', label='Oversold (Long)')
    axes[3].axhline(y=70, color='red', linestyle='--', label='Overbought (Short)')
    axes[3].fill_between(results_df.index, 0, 30, alpha=0.2, color='green')
    axes[3].fill_between(results_df.index, 70, 100, alpha=0.2, color='red')
    axes[3].set_title('RSI Indicators (Phase 10: >70 = Short Zone)')
    axes[3].set_ylabel('RSI')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    # Volatility
    axes[4].plot(results_df.index, results_df['volatility'] * 100, label='ATR %', color='purple')
    axes[4].axhline(y=2, color='green', linestyle='--', label='Greed (<2%)')
    axes[4].axhline(y=5, color='red', linestyle='--', label='Bear (>5%)')
    axes[4].fill_between(results_df.index, 5, results_df['volatility'].max() * 100, alpha=0.2, color='red')
    axes[4].set_title('Volatility (ATR %) - Phase 10: >5% = Bear Mode Eligible')
    axes[4].set_ylabel('ATR %')
    axes[4].legend()
    axes[4].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    initial_value = results_df['portfolio_value'].iloc[0]
    final_value = results_df['portfolio_value'].iloc[-1]
    max_value = results_df['portfolio_value'].max()
    min_value = results_df['portfolio_value'].min()
    pnl = final_value - initial_value
    roi = (pnl / initial_value) * 100
    max_dd = (max_value - min_value) / max_value * 100
    
    print(f"\n" + "="*60)
    print(f"PHASE 10 BEAR PROFIT STRATEGY RESULTS")
    print(f"="*60)
    print(f"Initial Value:  ${initial_value:.2f}")
    print(f"Final Value:    ${final_value:.2f}")
    print(f"Total P&L:      ${pnl:+.2f} ({roi:+.1f}%)")
    print(f"Max Value:      ${max_value:.2f}")
    print(f"Min Value:      ${min_value:.2f}")
    print(f"Max Drawdown:   {max_dd:.1f}%")
    print(f"")
    print(f"Time in Offensive (Long): {(results_df['mode'] == 'offensive').mean()*100:.1f}%")
    print(f"Time in Bear (Short):     {(results_df['mode'] == 'bear').mean()*100:.1f}%")
    print(f"Time in Defensive:        {(results_df['mode'] == 'defensive').mean()*100:.1f}%")

## 5. Short Position Analysis

In [ ]:
if 'actions_log' in dir() and len(actions_log) > 0:
    actions_df = pd.DataFrame(actions_log)
    
    print("\nAction Summary:")
    print(actions_df['action'].value_counts())
    
    print("\nMode Distribution:")
    print(actions_df['mode'].value_counts())
    
    print("\nBear Mode Actions (Shorts):")
    bear_actions = actions_df[actions_df['mode'] == 'bear']
    if len(bear_actions) > 0:
        print(f"  Count: {len(bear_actions)}")
        print(bear_actions.to_string())
    else:
        print("  No bear mode actions triggered")
    
    print("\nOffensive Actions (Longs):")
    offensive = actions_df[actions_df['mode'] == 'offensive']
    if len(offensive) > 0:
        print(f"  Count: {len(offensive)}")
        print(offensive['trigger'].value_counts())
    
    print("\nLeveraged Trades:")
    lev_trades = actions_df[actions_df['leverage'] > 0]
    print(f"  Count: {len(lev_trades)}")
    if len(lev_trades) > 0:
        print(f"  Avg leverage: {lev_trades['leverage'].mean():.1f}x")
        print(f"  Max leverage: {lev_trades['leverage'].max():.0f}x")
else:
    print("\nNo actions executed")

## 6. Benchmark Comparison (Phase 9 vs Phase 10)

In [ ]:
if 'XRP/USDT' in data and 'results_df' in dir():
    df = data['XRP/USDT']
    
    # Buy and hold benchmarks
    initial_xrp = df['close'].iloc[100]
    final_xrp = df['close'].iloc[-1]
    xrp_return = (final_xrp - initial_xrp) / initial_xrp * 100
    
    if 'BTC/USDT' in data:
        btc_df = data['BTC/USDT']
        initial_btc = btc_df['close'].iloc[100]
        final_btc = btc_df['close'].iloc[-1]
        btc_return = (final_btc - initial_btc) / initial_btc * 100
    else:
        btc_return = 0
    
    # Our strategy return
    strategy_return = roi if 'roi' in dir() else 0
    
    # Phase 9 reference (from previous backtest)
    phase9_return = -9.2  # From previous backtest
    
    print(f"\n" + "="*60)
    print(f"BENCHMARK COMPARISON")
    print(f"="*60)
    print(f"XRP Buy & Hold:     {xrp_return:+.1f}%")
    print(f"BTC Buy & Hold:     {btc_return:+.1f}%")
    print(f"Phase 9 Strategy:   {phase9_return:+.1f}%")
    print(f"Phase 10 Strategy:  {strategy_return:+.1f}%")
    print(f"")
    print(f"Phase 10 vs Phase 9: {strategy_return - phase9_return:+.1f}%")
    print(f"Phase 10 vs XRP:     {strategy_return - xrp_return:+.1f}%")
    print(f"Phase 10 vs BTC:     {strategy_return - btc_return:+.1f}%")
    
    # Performance assessment
    if strategy_return > phase9_return:
        improvement = strategy_return - phase9_return
        print(f"\n[SUCCESS] Phase 10 improved by {improvement:+.1f}% over Phase 9")
        if strategy_return >= 0:
            print(f"[GOAL MET] Flipped Phase 9's -9.2% to positive!")
    else:
        print(f"\n[REVIEW] Bear strategy needs tuning")

## 7. Short P&L Simulation

In [ ]:
# Simulate potential short profits on identified bear signals
if 'bear_df' in dir() and len(bear_df) > 0:
    print("\nSimulated Short P&L (if all bear signals were traded):")
    
    # Assume 5x leverage, 8% collateral
    collateral = 1000 * 0.08  # $80 per trade
    leverage = 5
    
    total_pnl = 0
    wins = 0
    losses = 0
    
    for _, row in bear_df.iterrows():
        # P&L = collateral * leverage * drop_pct
        pnl = collateral * leverage * (row['drop_pct'] / 100)
        total_pnl += pnl
        if pnl > 0:
            wins += 1
        else:
            losses += 1
    
    print(f"  Total trades: {len(bear_df)}")
    print(f"  Wins: {wins}, Losses: {losses}")
    print(f"  Win rate: {wins/len(bear_df)*100:.1f}%")
    print(f"  Total P&L: ${total_pnl:+.2f}")
    print(f"  Avg P&L per trade: ${total_pnl/len(bear_df):+.2f}")
    
    # Impact on Phase 9 result
    phase9_pnl = -204.78  # From previous backtest
    combined_pnl = phase9_pnl + total_pnl
    print(f"\n  Phase 9 P&L: ${phase9_pnl:+.2f}")
    print(f"  + Short profits: ${total_pnl:+.2f}")
    print(f"  = Combined P&L: ${combined_pnl:+.2f}")
    
    if combined_pnl > 0:
        print(f"\n  [SUCCESS] Bear profits would have flipped Phase 9 to positive!")

## 8. Next Steps

1. **Retrain PPO**: Run 2M timesteps with Phase 10 short rewards
2. **Live Paper**: Start `python src/live_paper.py --interval 15`
3. **Monitor**: Check logs/trades.csv for bear mode triggers
4. **Tune**: Adjust BEAR_CONFIDENCE_THRESHOLD if too aggressive/passive

In [ ]:
print("\n" + "="*60)
print("PHASE 10 BACKTEST COMPLETE")
print("="*60)
print("")
print("To retrain with short rewards:")
print("  python src/main.py --mode train-rl --timesteps 2000000 --device cuda")
print("")
print("To start live paper trading:")
print("  python src/live_paper.py --interval 15")
print("")
print("Key Phase 10 thresholds:")
print(f"  Bear mode: ATR > 5% + RSI > 70 + confidence > 82%")
print(f"  Short leverage: 3-5x (capped)")
print(f"  Max short exposure: 20% of portfolio")
print(f"  Short decay: Close after 14 days")